Definimos la clase nodo, que contendrá la información básica que requiere este modelo

In [34]:
class Node:
    def __init__(self, pos_x: int, pos_y: int, cost: int, parent=None) -> None:
        self.pos_x: int = pos_x
        self.pos_y: int = pos_y
        self.cost: int = cost
        self.parent = parent

Definimos la función Manhattan para calcular distancias entre dos nodos

In [35]:

def manhattan(current: Node, next: Node):
    return abs(current.pos_x - next.pos_x) + abs(current.pos_y - next.pos_y)

La función: `{python} get_children(node: Node, map: list[list[int]]) -> list[Node]` retorna una lista de nodos hijos, calculados a partir de la posición (x, y) del nodo que recibe como parámetro y evaluando la validez de las posiciónes circundantes a este.


In [36]:
def get_children(node: Node, map: list[list[int]]) -> list[Node]:
    neighbors = []
    directions = [
        (-1, 0),
        (1, 0),
        (0, -1),
        (0, 1),
    ]

    for offset_x, offset_y in directions:
        pos_x, pos_y = node.pos_x + offset_x, node.pos_y + offset_y
        if (
            0 <= pos_x < len(map)
            and 0 <= pos_y < len(map[0])
            and map[pos_x][pos_y] == 0
        ):
            neighbor: Node = Node(pos_x, pos_y, 0, node)
            neighbor.cost = node.cost + 1
            neighbors.append(neighbor)

    return neighbors

Creamos la función de búsqueda que recibe los dos nodos y el mapa (matríz de 0's y 1's)

Esta función crea una lista con los nodos por vistar y un set de tuplas (estructura de datos que no permite duplicados).

Cada iteración se realizan lo siguientes pasos:

1. ordena la lista de nodos de mayor a menor costo.
2. obtiene el nodo de menor costo.
3. evalua si el la posición del nodo actual coincide con el que buscamos:
    a. En caso de coincidir con el nodo buscado, se agrega la tupla de su posición  a una lista de salida.
    b. se obtiene el nodo padre del nodo actual para la siguiente iteración.
    c. el ciclo se rompe al encontrár el nodo origen ya que este no tendrá definido un "padre".
    d. se retorna la lista con las coordenadas obtenidas para el camino del punto origen al destino.
4. se agrega el nodo actual en el set de visitados.
5. con base en el nodo actual, se calculan los nodos hijos (arriba, abajo, izquierda y derecha)
5. al recorrer cada nodo hijo, evaluamos si ya lo hemos visitado anteriormente
6. en caso de no haber visitado aún el nodo y si tiene un costo menor al costo definido por la heurística manhattan entre el origen y el destino, se añade el nodo a la lista "por visitar"
7. En caso de no encontrar un camino posible, se retorna una lista vacía.

In [37]:
def search_path(start: Node, goal: Node, map: list[list[int]]):
    to_be_visited = [start]
    visited = set()

    while to_be_visited:
        to_be_visited.sort(key=lambda x: x.cost, reverse=True)
        current = to_be_visited.pop()

        if current.pos_x == goal.pos_x and current.pos_y == goal.pos_y:
            path = []
            while current:
                path.append((current.pos_x, current.pos_y))
                current = current.parent
            return path[::-1]

        visited.add((current.pos_x, current.pos_y))

        for child in get_children(current, map):
            if (child.pos_x, child.pos_y) in visited:
                continue

            if child.cost <= goal.cost:
                to_be_visited.append(child)

    return []

La función "move", recibe el nodo robot, el nodo paquete, el nodo meta del paquete y el mapa (matríz de 0's y 1's) realiza los pasos siguientes:

1. Calcula la función manhattan entre el robot y el paquete.
2. Obtiene la lista de coordenadas a seguir para alcanzar el paquete e imprime la ruta.
3. Calcula la función manhattan entre el paquete y el destino reiniciando el costo del paquete a 0 ya que este es nuestro nuevo punto de partida.
4. Obtiene la lista de coordenadas a seguir para alcanzar la meta e imprime la ruta.

In [39]:
def move(robot: Node, package: Node, goal: Node, map: list[list[int]], number: int):
    package.cost = manhattan(robot, package)
    for x, y in search_path(robot, package, map):
        print("mover R fila {}, columna {}".format(x, y))

    package.cost = 0
    goal.cost = manhattan(package, goal)
    for x, y in search_path(package, goal, map):
        print("cargar R M{} fila {}, columna {}".format(number, x, y))
    print("")


Por último definimos nuestra función "main" en donde inicializamos nuestro mapa, creamos nuestro nodo de posición para el robot, la lista de nodos de los paquetes y la lista de las metas.

A continuación iteramos los paquetes y las metas para calcular la ruta robot - paquete, paquete - meta y al finalizar reiniciamos la posición del robor en (2,2)


In [40]:


def main():
    map: list[tuple[int]] = [
        (0, 1, 0, 0), 
        (0, 1, 0, 0), 
        (0, 0, 0, 0), 
        (0, 0, 0, 0)
        ]

    robot: Node = Node(2, 2, 0)
    packages: list[Node] = [Node(0, 0, 0), Node(2, 0, 0), Node(0, 3, 0)]
    goals: list[None] = [Node(3, 3, 0), Node(3, 2, 0), Node(3, 1, 0)]

    for i in range(len(packages)):
        move(robot, packages[i], goals[i], map, i + 1)
        robot.pos_x = 2
        robot.pos_y = 2


main()

mover R fila 2, columna 2
mover R fila 2, columna 1
mover R fila 2, columna 0
mover R fila 1, columna 0
mover R fila 0, columna 0
cargar R M1 fila 0, columna 0
cargar R M1 fila 1, columna 0
cargar R M1 fila 2, columna 0
cargar R M1 fila 3, columna 0
cargar R M1 fila 3, columna 1
cargar R M1 fila 3, columna 2
cargar R M1 fila 3, columna 3

mover R fila 2, columna 2
mover R fila 2, columna 1
mover R fila 2, columna 0
cargar R M2 fila 2, columna 0
cargar R M2 fila 2, columna 1
cargar R M2 fila 3, columna 1
cargar R M2 fila 3, columna 2

mover R fila 2, columna 2
mover R fila 2, columna 3
mover R fila 1, columna 3
mover R fila 0, columna 3
cargar R M3 fila 0, columna 3
cargar R M3 fila 0, columna 2
cargar R M3 fila 1, columna 2
cargar R M3 fila 2, columna 2
cargar R M3 fila 3, columna 2
cargar R M3 fila 3, columna 1

